### Gonzalo Alvis

Usar urbanpy para descargar datos de una ciudad 

### Librerias

- urbanpy
- plotly
- geopandas

In [ ]:
import warnings
import sys
import urbanpy as up
import pandas as pd
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm.notebook import tqdm

sys.path.append("..")
warnings.filterwarnings("ignore") 
tqdm.pandas()

### Mapa

In [ ]:
ciudad = up.download.nominatim_osm(
    "Barcelona, España"
)  # expected_position is 0 by default

In [ ]:
ciudad.plot()

plt.show()

### population data

In [ ]:
city_resources=up.download.search_hdx_dataset("Spain")
city_resources

In [ ]:
pop_spn=up.download.get_hdx_dataset(city_resources,0)
pop_spn.head()

### Conversion a puntos y hexagonos

In [ ]:
pop_ba=up.geom.filter_population(pop_spn,ciudad)

In [ ]:
pop_ba.plot()

In [ ]:
hex_ba=up.geom.gen_hexagons(resolution=9,city=ciudad)
hex_ba.shape

### Mergeando layers

In [ ]:
pop_ba.head()

In [ ]:
hex_ba=up.geom.merge_shape_hex(hex_ba,pop_ba,agg={"esp_general_2020":"sum"})

In [ ]:
hex_ba.plot(
    "esp_general_2020", legend=True, missing_kwds={"color": "grey"}
)
plt.title("Barcelona - H3 res: 9")
plt.axis(False)
plt.tight_layout()

In [ ]:
fs, metadata = up.download.overpass(
    type_of_data="node",
    mask=ciudad,
    query={
        "amenity": ["marketplace"],
        "shop": [
            "supermarket",
            "kiosk",
            "mall",
            "convenience",
            "butcher",
            "greengrocer",
        ],
    },
)

In [ ]:
fs.shape

In [ ]:
up.routing.start_osrm_server("spain", "europe", "foot")

In [ ]:
hex_barcelona_access = up.accessibility.travel_times(hex_ba, fs, "food_facility")
hex_barcelona_access.head()

In [ ]:
up.routing.stop_osrm_server("spain", "europe", "foot")

### Interactive map

In [ ]:
plotly.offline.init_notebook_mode()

In [ ]:
fig = up.plotting.choropleth_map(
    hex_ba,
    "esp_general_2020",
    title="Estimated Population - 2020",
    width=400,
    height=400,
)

# Make space for the title
fig.update_layout(margin=dict(l=0, r=0, b=0))

fig.show()

In [ ]:
# Get ordered category labels
category_orders = (
    hex_barcelona_access["duration_to_nearest_food_facility_label"].unique().sort_values()
)
category_orders.categories

In [ ]:
hex_barcelona_access_filtered_pop = hex_barcelona_access.query(
    "esp_general_2020 > 0"
).reset_index(drop=True)

In [ ]:
fig = up.plotting.choropleth_map(
    hex_barcelona_access_filtered_pop,
    color_column="duration_to_nearest_food_facility_label",
    color_discrete_sequence=px.colors.sequential.Plasma_r,
    category_orders={"duration_to_nearest_food_facility_label": category_orders},
    labels={"duration_to_nearest_food_facility_label": "Minutes"},
    title="Travel Time to Nearest Food Facility",
    width=400,
    height=400,
)

# Make space for the title
fig.update_layout(margin=dict(l=0, r=0, b=0))

# Remove the hexagon outlines to make the map clearer
fig.update_traces(marker_line_width=0)

fig.show()